In [1]:
import os
import csv
import pandas as pd
from DatetimeExtractor import *

claims_folder = os.path.join("..", "..", "datasets", "flights", "clean_flight")
groundtruth_folder = os.path.join("..", "..","datasets", "flights", "flight_truth")

all_claims_filename = "clean_flight.csv"
all_truth_filename = "flight_truth.csv"

all_claims_file = os.path.join(claims_folder, all_claims_filename)
all_groundtruth_file = os.path.join(groundtruth_folder, all_truth_filename)

In [2]:
header_claims = [
    "source",
    "flight_number",
    "scheduled_departure",
    "actual_departure",
    "departure_gate", 
    "scheduled_arrival",
    "actual_arrival",
    "arrival_gate",
]

In [3]:
def import_gate(gate):
    gate = gate.strip()
    gate = gate.upper()
    if gate == "-" or gate == "--" or gate == "" or gate == "?" or gate == "$" or gate == "NOT PROVIDED BY AIRLINE" or gate is None:
        gate = "-"
    return gate

In [4]:
if os.path.isfile(all_claims_file):
    os.remove(all_claims_file)

In [5]:
clean_claims = list()

# claims input files, ignore any other hidden file in the folder...
claims_files = [f for f in os.listdir(claims_folder) if not f.startswith('.')]

for file_name in claims_files:
    
    file_path = os.path.join(claims_folder, file_name)

    print("Importing: " + file_path)

    with open(file_path, "r", encoding="iso-8859-1") as f:
       
        reader = csv.reader(f, delimiter='\t')
        year, month, day, c = file_name.split("-")
        dt_ex = DatetimeExtractor(int(year), int(month), int(day))

        for i, row in enumerate(reader):
            try:
                source = row[header_claims.index("source")].strip().upper()
                flight = row[header_claims.index("flight_number")].strip().upper()
                departure_gate = import_gate(row[header_claims.index("departure_gate")])  # one hot
                arrival_gate = import_gate(row[header_claims.index("arrival_gate")])  # one hot
                scheduled_departure = dt_ex.get_datetime(row[header_claims.index("scheduled_departure")])
                scheduled_arrival = dt_ex.get_datetime(row[header_claims.index("scheduled_arrival")])
                actual_departure = dt_ex.get_datetime(row[header_claims.index("actual_departure")])
                actual_arrival = dt_ex.get_datetime(row[header_claims.index("actual_arrival")])    
            except Exception as p:
                print(p)
            
            record = [source, flight, scheduled_departure, actual_departure, departure_gate, scheduled_arrival, actual_arrival, arrival_gate]
            clean_claims.append(record)


Importing: ../../datasets/flights/clean_flight/2011-12-10-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-29-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-14-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-08-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-27-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-30-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-09-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-11-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-19-data.txt
Importing: ../../datasets/flights/clean_flight/2012-01-02-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-16-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-31-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-02-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-07-data.txt
Importing: ../../datasets/flights/clean_flight/2011-12-28-data

In [6]:
claims = pd.DataFrame(clean_claims)
claims.columns = header_claims
claims.head(5)

,source,flight_number,scheduled_departure,actual_departure,departure_gate,scheduled_arrival,actual_arrival,arrival_gate
0,UA,UA-1534-RTB-IAH,2011-12-10 14:05:00,2011-12-10 17:00:00,-,2011-12-10 17:01:00,2011-12-10 19:44:00,E24
1,AIRTRAVELCENTER,UA-1534-RTB-IAH,NaT,2011-12-10 13:53:00,-,NaT,2011-12-10 18:18:00,-
2,MYRATEPLAN,UA-1534-RTB-IAH,NaT,2011-12-10 13:53:00,-,NaT,2011-12-10 18:18:00,-
3,HELLOFLIGHT,UA-1534-RTB-IAH,NaT,2011-12-10 13:53:00,-,NaT,2011-12-10 18:18:00,-
4,FLYTECOMM,UA-1534-RTB-IAH,NaT,2011-12-10 13:53:00,-,NaT,2011-12-10 18:18:00,-


In [7]:
claims.to_csv(all_claims_file, sep=",")

In [8]:
header_truth = [
    "flight_number",
    "scheduled_departure",
    "actual_departure",
    "departure_gate", 
    "scheduled_arrival",
    "actual_arrival",
    "arrival_gate",
]

In [9]:
if os.path.isfile(all_groundtruth_file):
    os.remove(all_groundtruth_file)

In [10]:
clean_truth = list()

groundtruth_files = [f for f in os.listdir(groundtruth_folder) if not f.startswith('.')]

for file_name in groundtruth_files:
    
    file_path = os.path.join(groundtruth_folder, file_name)

    print("Importing: " + file_path)

    with open(file_path, "r", encoding="iso-8859-1") as f:
        
        reader = csv.reader(f, delimiter='\t')
        year, month, day, c = file_name.split("-")
        dt_ex = DatetimeExtractor(int(year), int(month), int(day))

        for i, row in enumerate(reader):
            try:
                flight = row[header_truth.index("flight_number")].strip().upper()
                departure_gate = import_gate(row[header_truth.index("departure_gate")])
                arrival_gate = import_gate(row[header_truth.index("arrival_gate")])
                scheduled_departure = dt_ex.get_datetime(row[header_truth.index("scheduled_departure")])
                scheduled_arrival = dt_ex.get_datetime(row[header_truth.index("scheduled_arrival")])
                actual_departure = dt_ex.get_datetime(row[header_truth.index("actual_departure")])
                actual_arrival = dt_ex.get_datetime(row[header_truth.index("actual_arrival")])    
            except Exception as p:
                print(p)
            
            record = [flight, scheduled_departure, actual_departure, departure_gate, scheduled_arrival, actual_arrival, arrival_gate]
            clean_truth.append(record)

Importing: ../../datasets/flights/flight_truth/2011-12-08-truth.txt
Importing: ../../datasets/flights/flight_truth/2012-01-02-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-25-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-15-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-14-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-31-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-03-truth.txt
Importing: ../../datasets/flights/flight_truth/2012-01-01-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-04-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-19-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-28-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-24-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-18-truth.txt
Importing: ../../datasets/flights/flight_truth/2011-12-29-truth.txt
Importing: ../../datasets/flights/flight_truth/2

In [11]:
groundtruth = pd.DataFrame(clean_truth)
groundtruth.columns = header_truth
groundtruth.head(5)

,flight_number,scheduled_departure,actual_departure,departure_gate,scheduled_arrival,actual_arrival,arrival_gate
0,AA-1221-MCO-ORD,2011-12-08 20:00:00,2011-12-08 19:52:00,16,2011-12-08 21:50:00,2011-12-08 21:24:00,H11B
1,AA-4307-ORD-DTW,2011-12-08 18:45:00,2011-12-08 18:37:00,H2,2011-12-08 21:10:00,2011-12-08 20:57:00,D28
2,AA-616-DFW-DTW,2011-12-08 09:05:00,2011-12-08 09:00:00,C24,2011-12-08 12:35:00,2011-12-08 12:20:00,D32
3,AA-431-MIA-SFO,2011-12-08 08:35:00,2011-12-08 08:31:00,D22,2011-12-08 11:50:00,2011-12-08 11:31:00,57
4,AA-3756-ORD-SLC,2011-12-08 12:15:00,2011-12-08 12:12:00,H4,2011-12-08 14:45:00,2011-12-08 14:35:00,A3


In [12]:
groundtruth.to_csv(all_groundtruth_file, sep=",")

In [13]:
# groundtruth -> pandas dataframe with the groundtruth
# claims -> pandas dataframe with all the claims
# claims = pd.read_csv(all_claims_file)
# groundtruth = pd.read_csv(all_groundtruth_file)

#     Source    Object  Property   Value   Categorical
# 0     A         o1      p1         b          1
# 1     B         o1      p1         a          1
# 2     C         o1      p1         a          1
# 3     A         o2      p2         2          0
# 4     B         o2      p2         1          0
# 5     C         o2      p2         1          0
# 6     A         o3      p3         a          1

In [40]:
transformed_claims = list()

In [41]:
header_trans_claims = ["Source", "Object", "Property", "Value", "Categorical"]

In [42]:
from datetime import datetime

def get_datetime_minutes(date_to_convert):
    if pd.isnull(date_to_convert):
        date_to_convert = datetime(year=2012, month=12, day=1)
    start_date = datetime(year=2011, month=12, day=1)
    delta = date_to_convert - start_date
    delta_minutes = round(delta.total_seconds() / 60, 3)
    return delta_minutes

In [43]:
# header_claims = [
#    "source",
#    "flight_number",
#    "scheduled_departure",
#    "actual_departure",
#    "departure_gate", 
#    "scheduled_arrival",
#    "actual_arrival",
#    "arrival_gate",
# ]

print("Claims to transform : %i " %len(claims))

for i, el in claims.iterrows():
    
    sd = [el["source"], el["flight_number"], "scheduled_departure", get_datetime_minutes(el["scheduled_departure"]), 0]
    ad = [el["source"], el["flight_number"], "actual_departure", get_datetime_minutes(el["actual_departure"]), 0]
    dg = [el["source"], el["flight_number"], "departure_gate", el["departure_gate"], 1]
    sa = [el["source"], el["flight_number"], "scheduled_arrival", get_datetime_minutes(el["scheduled_arrival"]), 0]
    aa = [el["source"], el["flight_number"], "actual_arrival", get_datetime_minutes(el["actual_arrival"]), 0]
    ag = [el["source"], el["flight_number"], "arrival_gate", el["arrival_gate"], 1]
    
    transformed_claims.extend([sd, ad, dg, sa, aa, ag])
    
    if int( (i)/len(claims) * 100 ) < int( (i+1)/len(claims) * 100 ) and int( (i+1)/len(claims) * 100 ) % 10 == 0:
        print(int( (i+1)/len(claims) * 100 ), "% done...")

Claims to transform : 776067 
10 % done...
20 % done...
30 % done...
40 % done...
50 % done...
60 % done...
70 % done...
80 % done...
90 % done...
100 % done...


In [45]:
header_transformed_claims = ["Source", "Object", "Property", "Value", "Categorical"]
pd_transformed_claims = pd.DataFrame(transformed_claims)
pd_transformed_claims.columns = header_transformed_claims

In [48]:
pd_transformed_claims.head(10)

,Source,Object,Property,Value,Categorical
0,UA,UA-1534-RTB-IAH,scheduled_departure,13805,0
1,UA,UA-1534-RTB-IAH,actual_departure,13980,0
2,UA,UA-1534-RTB-IAH,departure_gate,-,1
3,UA,UA-1534-RTB-IAH,scheduled_arrival,13981,0
4,UA,UA-1534-RTB-IAH,actual_arrival,14144,0
5,UA,UA-1534-RTB-IAH,arrival_gate,E24,1
6,AIRTRAVELCENTER,UA-1534-RTB-IAH,scheduled_departure,527040,0
7,AIRTRAVELCENTER,UA-1534-RTB-IAH,actual_departure,13793,0
8,AIRTRAVELCENTER,UA-1534-RTB-IAH,departure_gate,-,1
9,AIRTRAVELCENTER,UA-1534-RTB-IAH,scheduled_arrival,527040,0


In [51]:
# header_truth = [
#    "flight_number",
#    "scheduled_departure",
#    "actual_departure",
#    "departure_gate", 
#    "scheduled_arrival",
#    "actual_arrival",
#    "arrival_gate",
# ]

transformed_truth = list()
header_transformed_truth = ["Object", "Property", "Value", "Categorical"]

print("Truth to transform : %i " %len(groundtruth))

for i, el in groundtruth.iterrows():
    
    sd = [el["flight_number"], "scheduled_departure", get_datetime_minutes(el["scheduled_departure"]), 0]
    ad = [el["flight_number"], "actual_departure", get_datetime_minutes(el["actual_departure"]), 0]
    dg = [el["flight_number"], "departure_gate", el["departure_gate"], 1]
    sa = [el["flight_number"], "scheduled_arrival", get_datetime_minutes(el["scheduled_arrival"]), 0]
    aa = [el["flight_number"], "actual_arrival", get_datetime_minutes(el["actual_arrival"]), 0]
    ag = [el["flight_number"], "arrival_gate", el["arrival_gate"], 1]
    
    transformed_truth.extend([sd, ad, dg, sa, aa, ag])
    
    if int( (i)/len(groundtruth) * 100 ) < int( (i+1)/len(groundtruth) * 100 ) and int( (i+1)/len(groundtruth) * 100 ) % 10 == 0:
        print(int( (i+1)/len(groundtruth) * 100 ), "% done...")

Truth to transform : 2986 
10 % done...
20 % done...
30 % done...
40 % done...
50 % done...
60 % done...
70 % done...
80 % done...
90 % done...
100 % done...


In [52]:
pd_transformed_truth = pd.DataFrame(transformed_truth)
pd_transformed_truth.columns = header_transformed_truth

In [53]:
pd_transformed_truth.head(10)

,Object,Property,Value,Categorical
0,AA-1221-MCO-ORD,scheduled_departure,11280,0
1,AA-1221-MCO-ORD,actual_departure,11272,0
2,AA-1221-MCO-ORD,departure_gate,16,1
3,AA-1221-MCO-ORD,scheduled_arrival,11390,0
4,AA-1221-MCO-ORD,actual_arrival,11364,0
5,AA-1221-MCO-ORD,arrival_gate,H11B,1
6,AA-4307-ORD-DTW,scheduled_departure,11205,0
7,AA-4307-ORD-DTW,actual_departure,11197,0
8,AA-4307-ORD-DTW,departure_gate,H2,1
9,AA-4307-ORD-DTW,scheduled_arrival,11350,0


In [61]:
print("Total Claims: ", len(pd_transformed_claims))

Total Claims:  4656402


In [62]:
print("Ground Truth: ", len(pd_transformed_truth))

Ground Truth:  17916
